In [1]:
#Import libraries
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geopy.extra.rate_limiter import RateLimiter
from geopy.point import Point

import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
#reads in National Address Database data for the Bronx (36005) - 'bronx_data'
file_path = '../dspg23_reverse_geocoder/Data/36005.csv'
bronx_data = pd.read_csv(file_path)

In [3]:
#Creates Reverse geocoder tool

#Input: df w/ a 'latitude' and 'longitude' column, Output: df with updated 'address' column

def reverse_geocode_tool(df):
    #Creates Geo column: combines latitude and longitude 
    df["Geo"] = df["latitude"].astype(str)+ ',' + df["longitude"].astype(str)
    
    geolocator = Nominatim(user_agent="reverse_geocoder", timeout=10)
    
    rgeocode = RateLimiter(geolocator.reverse, min_delay_seconds=0.001)
    
    #Reverse geocodes 'Geo' column and populates 'address' column with results
    df['address'] = df['Geo'].apply(rgeocode)
    
    return df

In [4]:
#'Missing_data': df containing all the rows of 'bronx_data' that have missing values in 'address' columns
missing_data = bronx_data[bronx_data['address'].isna()]

In [5]:
#Reads in shape file - 'bronx_shape'
shape_file_path = '../dspg23_reverse_geocoder/Data/tl_2020_36005_tabblock20/tl_2020_36005_tabblock20.shp'
bronx_shape = gpd.read_file(shape_file_path)

#Creates 'centroid' column based on 'geometry' - gives us latitude and longitude values
bronx_shape['centroid'] = bronx_shape['geometry'].centroid

#Converts 'GEOID20' column to int, so it can be merged with 'missing_data'
bronx_shape['GEOID20']=bronx_shape['GEOID20'].astype(int)

/var/folders/8z/5tmywm6s11d187fc5fg_fgb80000gn/T/ipykernel_22220/326869575.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  bronx_shape['centroid'] = bronx_shape['geometry'].centroid


In [6]:
#Merges 'missing_data' with 'bronx_shape' on 'GEOID20' col (left join)
merged_df = missing_data.merge(bronx_shape, on='GEOID20', how="left")
merged_df = merged_df.drop(['STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'BLOCKCE20', 'NAME20', 'MTFCC20', 'UR20', 'UACE20', 'UATYPE20', 'FUNCSTAT20', 'ALAND20', 'AWATER20', 'INTPTLAT20', 'INTPTLON20', 'geometry'], axis=1)

#Poulates 'latitude' and 'longitude' cols of 'merged_df' with lat and long vals from 'centroid'
for i in merged_df.index:   
    point = merged_df['centroid'][i]
    lat = point.y
    long = point.x
    
    merged_df['latitude'][i] = lat
    merged_df['longitude'][i] = long
    
merged_df = merged_df.drop('centroid', axis=1)

#Reverse geocodes 'merged_df'
reverse_geocode_tool(merged_df)
merged_df = merged_df.drop('Geo', axis=1)

/var/folders/8z/5tmywm6s11d187fc5fg_fgb80000gn/T/ipykernel_22220/2210636414.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['latitude'][i] = lat
/var/folders/8z/5tmywm6s11d187fc5fg_fgb80000gn/T/ipykernel_22220/2210636414.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['longitude'][i] = long


In [7]:
#Checks number of 'address' values that are NaN
#Should be 1008
sum(bronx_data['address'].isna())

1008

In [8]:
np.where(bronx_data['address'].isna())

(array([    76,    242,    264, ..., 105440, 105443, 106275]),)

In [9]:
#Checks if 'address' column in 'bronx_data' is NaN
#if so, updates 'address', 'latitude', and 'longitude' columns of 'bronx_data' 
#with 'address', 'latitude', and 'longitude' columns of 'test_data', based on 'GEOID20'

for i in bronx_data.index:
    if pd.isna(bronx_data['address'][i]):  
        bronx_data['address'][i] = merged_df['address'][merged_df.loc[merged_df['GEOID20']==bronx_data['GEOID20'][i]].index[0]]
        bronx_data['latitude'][i] = merged_df['latitude'][merged_df.loc[merged_df['GEOID20']==bronx_data['GEOID20'][i]].index[0]]
        bronx_data['longitude'][i] = merged_df['longitude'][merged_df.loc[merged_df['GEOID20']==bronx_data['GEOID20'][i]].index[0]]

/var/folders/8z/5tmywm6s11d187fc5fg_fgb80000gn/T/ipykernel_22220/1285047586.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bronx_data['address'][i] = merged_df['address'][merged_df.loc[merged_df['GEOID20']==bronx_data['GEOID20'][i]].index[0]]
/var/folders/8z/5tmywm6s11d187fc5fg_fgb80000gn/T/ipykernel_22220/1285047586.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bronx_data['latitude'][i] = merged_df['latitude'][merged_df.loc[merged_df['GEOID20']==bronx_data['GEOID20'][i]].index[0]]
/var/folders/8z/5tmywm6s11d187fc5fg_fgb80000gn/T/ipykernel_22220/1285047586.py:9: SettingWithCopyWarning: 
A value is trying to 

In [10]:
bronx_data

address          GEOID20  \
0       3001 henry hudson parkway west,bronx,ny,10463  360050301001001   
1       3053 henry hudson parkway west,bronx,ny,10463  360050301001001   
2       3051 henry hudson parkway west,bronx,ny,10463  360050301001001   
3       3055 henry hudson parkway west,bronx,ny,10463  360050301001001   
4       3057 henry hudson parkway west,bronx,ny,10463  360050301001001   
...                                               ...              ...   
106600             1122 pugsley avenue,bronx,ny,10472  360050078001003   
106601            2012 haviland avenue,bronx,ny,10472  360050078001003   
106602            2016 haviland avenue,bronx,ny,10472  360050078001003   
106603            2014 haviland avenue,bronx,ny,10472  360050078001003   
106604            2018 haviland avenue,bronx,ny,10472  360050078001003   

        longitude   latitude  
0      -73.916170  40.882786  
1      -73.915858  40.883261  
2      -73.915876  40.883211  
3      -73.915833  40.883307  
4      -73.915808  40.883353  
...           ...        ...  
106600 -73.856574  40.829606  
106601 -73.856259  40.829611  
106602 -73.856139  40.829627  
106603 -73.856200  40.829619  
106604 -73.856078  40.829635  

[106605 rows x 4 columns]

In [11]:
##Checks number of 'address' values that are NaN
#Should be 0
sum(bronx_data['address'].isna())

0

In [12]:
np.where(bronx_data['address'].isna())

(array([], dtype=int64),)

In [13]:
#Saves output df as csv
bronx_data.to_csv('../dspg23_reverse_geocoder/36005_updated.csv')